In [102]:
import numba
import numpy as np
from functools import reduce
from operator import mul

from minitorch.autodiff.tensor_data import to_index

In [2]:
def index_to_position(index, strides):
    position = 0
    for i, s in zip(index, strides):
        position += i * s
    return position


index_to_position = numba.njit(inline="always")(index_to_position)

In [58]:
shape = 10, 4
index = (5, 3)
strides = (4, 1)
index_to_position(index, strides)

23

In [32]:
def to_index_numba(ordinal: int, shape, out_index):
    shape = reversed(shape)
    for i in range(len(shape)):
        is_last_dim = i == (len(shape) - 1)
        if is_last_dim:
            out_index[i] = remaining_ordinal
        else:
            out_index[i] = int(remaining_ordinal // dim)
            remaining_ordinal = remaining_ordinal % dim

to_index_numba = numba.njit(inline="always")(to_index_numba)
    

In [120]:
shape = np.array((4, 4, 1))
ordinal = 14
out_index = np.array([0, 0, 0])
to_index(ordinal, shape, out_index)
out_index

array([3, 2, 0])

In [123]:
shape = np.array((4, 4, 1))
ordinal = 14
out_index = np.array([0, 0, 0])

# enter function
shape = shape[::-1]
for i, dim in enumerate(shape):
    if i == 0:
        div = 1
    else:
        div = np.prod(shape[:i])
    idx = (ordinal // div) % dim
    out_index[(len(out_index) - 1) - i] = idx

out_index

array([3, 2, 0])

In [111]:
shape[2:2:-1]

()

In [99]:
# Last index
last = 1 % 2

# Middle index
middle = (1 // (2)) % 3

# because product is now bigger than ordinal
first = (1 // (3*2)) % 5

(first, middle ,last)

(0, 0, 1)

In [100]:
l = [1, 2, 3]
for i in range(len(l)):
    if i == 0:
        

l[0]

1

In [60]:
shape = (5, 3)
ordinal = 10
out_index = np.array([0, 0])
to_index_numba(ordinal, shape, out_index)
out_index

array([3, 1])

In [42]:
def broadcast_index(big_index, big_shape, shape , out_index):
    for i in range(len(shape)):
        print(i)
        offset = i + len(big_shape) - len(shape)
        print(offset)
        out_index[i] = big_index[offset] if shape[i] != 1 else 0

In [56]:
big_shape = (1, 5, 5, 3)
big_index = (0, 0, 3, 1)
shape = (5, 1)
out_index = np.array([0, 0])

broadcast_index(big_index, big_shape, shape, out_index)
out_index


0
2
1
3


array([3, 0])

In [48]:
def to_numpy(*inputs):
    out = 
    return [np.array(in_) for in_ in inputs]

to_numpy = numba.njit(inline="always")(to_numpy)

In [49]:
shape1 = (1, 2, 3)
shape2 = (2, 3, 4)
shape3 = (3, 4, 5)
shape1, shape2, shape3 = to_numpy(shape1, shape2, shape3)

In [52]:
np.zeros(np.array([1, 3]))

array([[0., 0., 0.]])